In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import datetime as dt

from typing import List

In [2]:
hosps = pd.read_csv('data-ingest/data/hospitalisations_hb_long_unfiltered.csv', parse_dates = [0])
hosps.head()

,date,health_board,admission_type,count,covid_status
0,2020-04-01,Betsi Cadwaladr University Health Board,All,768,NaN
1,2020-04-01,Betsi Cadwaladr University Health Board,C19,107,All
2,2020-04-01,Betsi Cadwaladr University Health Board,C19_Con,29,Confirmed
3,2020-04-01,Betsi Cadwaladr University Health Board,C19_Sus,78,Suspected
4,2020-04-01,Betsi Cadwaladr University Health Board,NonC19,661,NaN


In [3]:
hosps_filtered = hosps[hosps['admission_type'].isin(['C19_Con', 'C19_Sus', 'C19_Rec'])].drop(columns = ['admission_type'])
hosps_filtered = hosps_filtered[hosps_filtered['date'] >= dt.datetime.strptime('2020-09-01', '%Y-%m-%d')]
hosps_filtered.tail()

,date,health_board,count,covid_status
11955,2020-11-19,Velindre University NHS Trust,0,Recovering
11956,2020-11-19,Velindre University NHS Trust,0,Suspected
11960,2020-11-19,Wales,1019,Confirmed
11961,2020-11-19,Wales,440,Recovering
11962,2020-11-19,Wales,187,Suspected


In [4]:
lockdowns = pd.read_csv('data-ingest/data/lockdown_status_hb_date.csv', parse_dates = [0])
lockdowns.tail()

,date,health_board,prop_lockdown,lockdown_status,proportion_lockdown
2070,2020-11-15,Wales,2.0,Post-firebreak,0.0
2071,2020-11-16,Wales,2.0,Post-firebreak,0.0
2072,2020-11-17,Wales,2.0,Post-firebreak,0.0
2073,2020-11-18,Wales,2.0,Post-firebreak,0.0
2074,2020-11-19,Wales,2.0,Post-firebreak,0.0


In [5]:
# we don't need the number 2 to distinguish between local/firebreak lockdowns. Replace it.
lockdowns['proportion_lockdown'] = lockdowns['prop_lockdown'].apply(lambda x: 100 if x==2 else int(round(x*100)))
#lockdowns = lockdowns.drop(columns = ['prop_lockdown'])
lockdowns.proportion_lockdown.unique()

array([  0,  63, 100,  46,  84,  72,  73,  54,  13], dtype=int64)

In [6]:
# join the tables
hosps_ld = hosps_filtered.merge(lockdowns, on = ['date', 'health_board'], how = 'inner')
hosps_ld.head()

,date,health_board,count,covid_status,prop_lockdown,lockdown_status,proportion_lockdown
0,2020-09-01,Betsi Cadwaladr University Health Board,1,Confirmed,0.0,No local lockdowns,0
1,2020-09-01,Betsi Cadwaladr University Health Board,26,Recovering,0.0,No local lockdowns,0
2,2020-09-01,Betsi Cadwaladr University Health Board,45,Suspected,0.0,No local lockdowns,0
3,2020-09-01,Hywel Dda University Health Board,2,Confirmed,0.0,No local lockdowns,0
4,2020-09-01,Hywel Dda University Health Board,0,Recovering,0.0,No local lockdowns,0


In [7]:
hosps_ld['label'] = hosps_ld['proportion_lockdown'].astype(str) + '% ' + hosps_ld['lockdown_status'].map(
    {'No local lockdowns': 'lockdown', 'Local lockdowns': 'local lockdown', 
    'Firebreak lockdown': 'firebreak', 'Post-firebreak': 'lockdown'}
)
hosps_ld.drop(columns = ['proportion_lockdown'], inplace = True)
hosps_ld.head()

,date,health_board,count,covid_status,prop_lockdown,lockdown_status,label
0,2020-09-01,Betsi Cadwaladr University Health Board,1,Confirmed,0.0,No local lockdowns,0% lockdown
1,2020-09-01,Betsi Cadwaladr University Health Board,26,Recovering,0.0,No local lockdowns,0% lockdown
2,2020-09-01,Betsi Cadwaladr University Health Board,45,Suspected,0.0,No local lockdowns,0% lockdown
3,2020-09-01,Hywel Dda University Health Board,2,Confirmed,0.0,No local lockdowns,0% lockdown
4,2020-09-01,Hywel Dda University Health Board,0,Recovering,0.0,No local lockdowns,0% lockdown


In [8]:
hosps_ld = hosps_ld[~hosps_ld['health_board'].isin(['Velindre University NHS Trust', 'Wales'])]

In [9]:
fig = px.bar(hosps_ld[hosps_ld['date'] == dt.datetime.strptime('2020-10-01', '%Y-%m-%d')],
            x = 'health_board', y = 'count', color = 'covid_status')
fig.show()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [10]:
import re

df = hosps_ld.copy()
df = df[df['health_board'] != 'Powys Teaching Health Board']
df['date'] = df['date'].astype(str)
df['health_board'] = df['health_board'].apply(lambda x: re.sub('(University)* Health Board', '', x))

max(df.groupby(['health_board', 'date'])['count'].sum())

493

In [21]:
color_map = dict(
    Suspected = '#747875',
    Confirmed = '#7d9caa',
    Recovering = '#9C528B'
)

fig = px.bar(df, animation_frame = 'date',
            x = 'health_board', y = 'count', color = 'covid_status',
            range_y = [0, 500], color_discrete_map = color_map,
            category_orders = {'covid_status': ['Suspected', 'Confirmed', 'Recovering']})
fig.update_layout(legend_traceorder = 'reversed', margin=dict(l=20, r=20, t=50, b=250))
fig.show()

Here I'm going to switch to plotly.graph_objects - I'm struggling to order the bars and place the slider using plotly express.

In [12]:
day = '2020-09-10'

# look at the categories we have - health board and lockdown status
today = df[df['date'] == day]
cats = today.groupby(['health_board'])[['health_board', 'lockdown_status']].first()
cats

,health_board,lockdown_status
health_board,,
Abertawe Bro Morgannwg,Abertawe Bro Morgannwg,No local lockdowns
Aneurin Bevan,Aneurin Bevan,Local lockdowns
Betsi Cadwaladr,Betsi Cadwaladr,No local lockdowns
Cardiff and Vale,Cardiff and Vale,No local lockdowns
Cwm Taf,Cwm Taf,No local lockdowns
Hywel Dda,Hywel Dda,No local lockdowns


In [30]:
title = "Number of people in hospital on "

color_map = dict(
    Suspected = '#747875',
    Confirmed = '#7d9caa',
    Recovering = '#9C528B'
)

frames = []
hover_template = '%{x[1]}: %{y}'

for day in df.date.unique():
    today = df[df['date'] == day]
    # extract categories (lockdown status will change over time)
    cats = today.groupby(['health_board'])[['health_board', 'lockdown_status']].first()
    # create 2d array for x axis - will revert to multicategory
    x = [['<br><br><br>' + stat for stat in cats['lockdown_status']], cats['health_board'].tolist()]
    # extract count of different covid patients
    y_s = today[today['covid_status']=='Suspected']['count']
    y_c = today[today['covid_status']=='Confirmed']['count']
    y_r = today[today['covid_status']=='Recovering']['count']
    # add them together for purpose of ordering bars
    y_tot = [s+c+r for s,c,r in zip(y_s, y_c, y_r)]
    # zip all these things together and sort by 1) descending total patient count, and 2) lockdown status
    zipped = zip(x[0], x[1], y_s, y_c, y_r, y_tot)
    sorted_list = sorted(sorted(zipped, key = lambda x: x[5], reverse=True), key = lambda x: x[0])
    # recreate 2d array for x axis
    new_x = [[l[0] for l in sorted_list], [l[1] for l in sorted_list]]
    # key to sorted list: 0 loc, 1 hb, 2 y_s, 3 y_c, 4 y_r, 5 y_tot 
    # create frame
    frame = go.Frame(
        data = [
            go.Bar(
            name = 'Suspected', x = new_x, y = [y[2] for y in sorted_list],
            marker_color = color_map['Suspected'], hovertemplate = hover_template),
            go.Bar(
            name = 'Confirmed', x = new_x, y = [y[3] for y in sorted_list],
            marker_color = color_map['Confirmed'], hovertemplate = hover_template),
            go.Bar(
            name = 'Recovering', x = new_x,y = [y[4] for y in sorted_list],
            marker_color = color_map['Recovering'], hovertemplate = hover_template)
        ],
        layout=go.Layout(font={'size': 14},
                             plot_bgcolor = '#FFFFFF',
                             bargap=0.15,
                             title=title + day,
                             yaxis = dict(range = [0,500]))
    )
    frames.append(frame)
    
# prepare data for day one
today = df[df['date'] == '2020-09-01']
cats = today.groupby(['health_board'])[['health_board', 'lockdown_status']].first()
x = [cats['lockdown_status'].tolist(), cats['health_board'].tolist()] 

# set up first day
fig = go.Figure(data = [go.Bar(
        name = 'Suspected', x = x,y = today[today['covid_status']=='Suspected']['count'],
        marker_color = color_map['Suspected']),
        go.Bar(
        name = 'Confirmed', x = x,y = today[today['covid_status']=='Confirmed']['count'],
        marker_color = color_map['Confirmed']),
        go.Bar(
        name = 'Recovering', x = x,y = today[today['covid_status']=='Recovering']['count'],
        marker_color = color_map['Recovering'])
        ], frames = frames,
            # add layout
            layout=go.Layout(font={'size': 14}, plot_bgcolor = '#FFFFFF',
                bargap=0.15, title=title + '2020-09-01',
                # add buttons
                updatemenus=[dict(type="buttons",
                buttons=[dict(label="Play",
                                method="animate",
                                args=[None,{"frame": {"duration": 800, "redraw": True}, 
                                            "fromcurrent": True, "transition": {"duration": 500,
                                                                                "easing": "cubic"}}]),
                            dict(label="Stop",
                                method="animate",
                                args=[[None],
                                      {"frame": {"duration": 0, "redraw": False}, 
                                       "mode": "immediate","transition": {"duration": 0}}
                                     ]
                                )
                        ]
                                 )
                            ])
               )
fig.update_layout(barmode = 'stack')
fig.write_html('visualisations/hospitalisations_health_boards_time.html')
fig.show()

In [31]:
# Sort df by health board alphabetically
df_fixed = df.sort_values(by = ['health_board', 'date'])
df_fixed

,date,health_board,count,covid_status,prop_lockdown,lockdown_status,label
6,2020-09-01,Abertawe Bro Morgannwg,0,Confirmed,0.0,No local lockdowns,0% lockdown
7,2020-09-01,Abertawe Bro Morgannwg,3,Recovering,0.0,No local lockdowns,0% lockdown
8,2020-09-01,Abertawe Bro Morgannwg,13,Suspected,0.0,No local lockdowns,0% lockdown
33,2020-09-02,Abertawe Bro Morgannwg,0,Confirmed,0.0,No local lockdowns,0% lockdown
34,2020-09-02,Abertawe Bro Morgannwg,3,Recovering,0.0,No local lockdowns,0% lockdown
...,...,...,...,...,...,...,...
2110,2020-11-18,Hywel Dda,1,Recovering,2.0,Post-firebreak,100% lockdown
2111,2020-11-18,Hywel Dda,11,Suspected,2.0,Post-firebreak,100% lockdown
2136,2020-11-19,Hywel Dda,113,Confirmed,2.0,Post-firebreak,100% lockdown
2137,2020-11-19,Hywel Dda,0,Recovering,2.0,Post-firebreak,100% lockdown


In [43]:
# RECREATING THE CHART WITHOUT SORTING

title = "Number of people in hospital on "

color_map = dict(
    Suspected = '#747875',
    Confirmed = '#7d9caa',
    Recovering = '#9C528B'
)

frames = []
hover_template = '%{x[1]}: %{y}'

for day in df.date.unique():
    today = df_fixed[df_fixed['date'] == day]
    x = today['health_board'].unique()
    y_data = {}
    # extract count of different covid patients
    y_data['Suspected'] = today[today['covid_status']=='Suspected']['count'].tolist()
    y_data['Confirmed'] = today[today['covid_status']=='Confirmed']['count'].tolist()
    y_data['Recovering'] = today[today['covid_status']=='Recovering']['count'].tolist()
    # add them together for purpose of ordering bars
    y_tot = [y_data['Recovering'][i] + y_data['Confirmed'][i] + y_data['Suspected'][i] for i in range(len(y_data['Recovering']))]
    # create frame
    frame = go.Frame(
        data = [
            go.Bar(
            name = stat, x = x, y = y_data[stat], marker_color = color_map[stat], hovertemplate = hover_template
            ) for stat in ['Suspected', 'Confirmed', 'Recovering']  
        ],
        layout=go.Layout(font={'size': 14},
                             plot_bgcolor = '#FFFFFF',
                             bargap=0.15,
                             title=title + day,
                             yaxis = dict(range = [0,500]))
    )
    frames.append(frame)
    
# prepare data for day one
today = df_fixed[df_fixed['date'] == '2020-09-01']
x = today['health_board']

# set up first day
fig = go.Figure(
    data = [go.Bar(
            name = stat, x = x, y = y_data[stat], marker_color = color_map[stat], hovertemplate = hover_template
            ) for stat in ['Suspected', 'Confirmed', 'Recovering']  
            ], 
            frames = frames,
            # add layout
            layout=go.Layout(font={'size': 14}, plot_bgcolor = '#FFFFFF',
                bargap=0.15, title=title + '2020-09-01',
                # add buttons
                updatemenus=[dict(type="buttons",
                buttons=[dict(label="Play",
                                method="animate",
                                args=[None,{"frame": {"duration": 800, "redraw": True}, 
                                            "fromcurrent": True, "transition": {"duration": 500,
                                                                                "easing": "cubic"}}]),
                            dict(label="Stop",
                                method="animate",
                                args=[[None],
                                      {"frame": {"duration": 0, "redraw": False}, 
                                       "mode": "immediate","transition": {"duration": 0}}
                                     ]
                                )
                        ]
                                 )
                            ])
               )

fig.update_layout(barmode = 'stack')
fig.write_html('visualisations/hospitalisations_health_boards_nosort.html')
fig.show()

In [39]:
y_data['Recovering']

1     26
4      0
7      3
10    39
13    19
16     9
Name: count, dtype: int64

In [44]:
first_day = '2020-09-01'

fig = go.Figure()
data_slider = []
test = []
hover_template = '%{x[1]}: %{y}'

for day in df.date.unique():
    today = df_fixed[df_fixed['date'] == day]
    x = today['health_board'].unique()
    y_data = {}
    # extract count of different covid patients
    y_data['Suspected'] = today[today['covid_status']=='Suspected']['count'].tolist()
    y_data['Confirmed'] = today[today['covid_status']=='Confirmed']['count'].tolist()
    y_data['Recovering'] = today[today['covid_status']=='Recovering']['count'].tolist()
    # add them together for purpose of ordering bars
    y_tot = [y_data['Recovering'][i] + y_data['Confirmed'][i] + y_data['Suspected'][i] for i in range(len(y_data['Recovering']))]
    test.append(zipped)
    for stat in ['Suspected', 'Confirmed', 'Recovering']:
        trace = go.Bar(
        name = stat, x = x, y = y_data[stat], 
        marker = dict(color = color_map[stat]), visible = False, hovertemplate = hover_template
        )
        fig.add_trace(trace)
        data_slider.append(trace)
    
for i in range(3):
    fig.data[i].visible = True

# Create and add slider
steps = []
i = 0
date_add = 0
while i < int(len(data_slider)): 
    new_date = str((dt.datetime.strptime(first_day, "%Y-%m-%d") + dt.timedelta(days = date_add)).date())
    step = dict(
        method="update",
        args=[{"visible": [False] * len(data_slider)},
              {"title": "Number of people in hospital on " + new_date}],
        label = new_date
    )
    for j in range(3):
        step["args"][0]["visible"][i+j] = True  # Toggle i'th trace to "visible"
    i+=3
    date_add += 1
    steps.append(step)
    
sliders = [dict(
    active=0,
    pad=dict(r= 10, t= 200),
    steps=steps
)]

# WANT TO GET THESE BUTTONS ONTO THE CHART, but how?!
update_layout_dicts =[{
    'type': 'buttons',
    'buttons':[
            {
                "args": [None, {"frame": {"duration": 800, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 500,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": True},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }]}
            ]


fig.update_layout(barmode = 'stack', 
                  sliders = sliders, 
                  margin=dict(l=20, r=20, t=50, b=300), 
                  yaxis = dict(range = [0,500]),
                  font = dict(size=16)
                  #xaxis = dict(title = dict(standoff=10))
                 # updatemenus = update_layout_dicts
                 )

fig.write_html('visualisations/TEST2.html')
fig.show()